In [1]:
import pandas as pd
import numpy as np
import openpyxl
import sys
import os
from pathlib import Path

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\utils")

from players_season_builder import *

In [2]:
folder_path = r"\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\Big Data Ball - Game" 
file_type, sheet_name = "xlsx" , 0
data_source = "BDB-GAME"


In [3]:
temp_df = combine_xl_data(folder_path, file_type, sheet_name, data_source) 

In [12]:
temp_df['TEAMS'].unique()
print(temp_df.columns)

temp_df.head()

Index(['DATASET', 'DATE', 'TEAMS', 'VENUE', '1Q', '2Q', '3Q', '4Q', 'OT1',
       'OT2', 'OT3', 'OT4', 'F', 'MIN', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA',
       'OR', 'DR', 'TOT', 'A', 'PF', 'ST', 'TO', 'BL', 'PTS', 'POSS', 'PACE',
       'OEFF', 'DEFF', 'REST DAYS'],
      dtype='object')


,DATASET,DATE,TEAMS,VENUE,1Q,2Q,3Q,4Q,OT1,OT2,...,PF,ST,TO,BL,PTS,POSS,PACE,OEFF,DEFF,REST DAYS
0,2006-2007 Regular Season,2006-10-31 00:00:00,Chicago,R,22,37,21,28,NaN,NaN,...,21,10,16,5,108,91.838947,91.838947,117.597167,71.864935,3+
1,2006-2007 Regular Season,2006-10-31 00:00:00,Miami,H,16,14,21,15,NaN,NaN,...,24,6,21,3,66,91.838947,91.838947,71.864935,117.597167,3+
2,2006-2007 Regular Season,2006-10-31 00:00:00,Phoenix,R,41,17,21,27,NaN,NaN,...,25,7,21,5,106,100.017714,100.017714,105.981226,113.979809,3+
3,2006-2007 Regular Season,2006-10-31 00:00:00,LA Lakers,H,26,27,34,27,NaN,NaN,...,19,11,20,1,114,100.017714,100.017714,113.979809,105.981226,3+
4,2006-2007 Regular Season,2006-11-01 00:00:00,Indiana,R,23,26,28,29,NaN,NaN,...,26,14,17,5,106,101.222117,101.222117,104.720197,97.804712,3+


Handle binary variables
- Rest Days:    
    '3+', 'B2B', 1, 2, '3IN4', '3IN4-B2B', '4IN5-B2B', '4IN5-B2B2B', '4IN5', '5IN6', 'B2B2B', '3IN4-B2B2B', '5IN6-B2B2B', '1', '2' '5IN5-B2B2B'

    Assumptions: 
        B2B = 0 days of rest
        Number of Days: 0, 1, 2, 3+
        Density of Games: How do we want to consider density

    # The more accurate method will be comparing dates and manually calculating 

- Applicable Data:

    Filter for "Regular Season"


In [5]:
# Filter for Regular Season
nba_regular_season = temp_df[temp_df['DATASET'].str.contains("Season")]
nba_regular_season['DATASET'] = nba_regular_season['DATASET'].str.replace(
    'NBA 2018-2019 Regular Season',
    '2018-2019 Regular Season'
)
nba_regular_season['DATASET'] = nba_regular_season['DATASET'].str.replace(
    'NBA 2019-2020 Regular Season',
    '2019-2020 Regular Season'
)
# Filter for playoffs
nba_playoffs = temp_df[temp_df['DATASET'].str.contains("Playoffs")]
nba_regular_season['DATASET'] = nba_regular_season['DATASET'].str.replace(
    'NBA 2020 Playoffs', '2020 Playoffs')
nba_regular_season['DATASET'] = nba_regular_season['DATASET'].str.replace(
    'NBA 2019 Playoffs','2019 Playoffs')


<ipython-input-5-7c42f2398efc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_regular_season['DATASET'] = nba_regular_season['DATASET'].str.replace(
<ipython-input-5-7c42f2398efc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_regular_season['DATASET'] = nba_regular_season['DATASET'].str.replace(
<ipython-input-5-7c42f2398efc>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [6]:
# Address Types
nba_regular_season['REST DAYS'] = nba_regular_season['REST DAYS'].astype('string')
nba_regular_season['DATE'] = pd.to_datetime(
    nba_regular_season['DATE']
    )

<ipython-input-6-0f00ec012475>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_regular_season['REST DAYS'] = nba_regular_season['REST DAYS'].astype('string')
<ipython-input-6-0f00ec012475>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_regular_season['DATE'] = pd.to_datetime(


In [7]:
nba_regular_season['TEAMS'].unique()

array(['Chicago', 'Miami', 'Phoenix', 'LA Lakers', 'Indiana', 'Charlotte',
       'Orlando', 'Atlanta', 'Philadelphia', 'New Orleans', 'Boston',
       'Toronto', 'New Jersey', 'Washington', 'Cleveland', 'Milwaukee',
       'Detroit', 'New York', 'Memphis', 'Sacramento', 'Minnesota',
       'Houston', 'Utah', 'Portland', 'Seattle', 'Golden State',
       'LA Clippers', 'San Antonio', 'Dallas', 'Denver', 'Oklahoma City',
       'Brooklyn'], dtype=object)

In [18]:
# Handle Rest Day - Recalculation
test = list()
for season in nba_regular_season['DATASET'].unique():
    for team in list(nba_regular_season[nba_regular_season['DATASET'] == season]['TEAMS'].unique()):
        temp_season = nba_regular_season.loc[nba_regular_season['DATASET'].eq(season) & nba_regular_season['TEAMS'].eq(team) ].sort_values(by=['TEAMS', 'DATE'])
        temp_season['DATE-DIFF'] = temp_season['DATE'].diff().dt.days
        temp_season['H/R CHANGE'] = temp_season['VENUE'].shift(1) != temp_season['VENUE'] # H --> R or R --> H 
        # Not sure how to handle multiple Road games
        temp_season['RR VAL'] = temp_season['VENUE'].shift(1) + temp_season['VENUE']
        temp_season['RR SERIES'] = ( temp_season['VENUE'].shift(1) + temp_season['VENUE']) == "RR"
        test.append(temp_season)

nba_regular_season = pd.concat(test)

# Date Diff
nba_regular_season.loc[nba_regular_season['DATE-DIFF'] > 14, ['DATE-DIFF']] = '14+'
nba_regular_season.loc[nba_regular_season['DATE-DIFF'].isna(), ['DATE-DIFF']] = 'Season Start'
nba_regular_season['DATE-DIFF'] = nba_regular_season['DATE-DIFF'].astype(str)
nba_regular_season['DATE-DIFF'] = nba_regular_season['DATE-DIFF']+" days"

# Travel - 
# LA Lakers, LA Clippers
# New York, Brooklyn
# 
# nba_regular_season['SAME CITY'] = 0
# nba_regular_season.loc[nba_regular_season['OWN TEAM'].eq("LA Lakers") & nba_regular_season['OPP TEAM'].eq("LA Clippers"), ['SAME CITY'] ] = 1
# nba_regular_season.loc[nba_regular_season['OWN TEAM'].eq("LA Clippers") & nba_regular_season['OPP TEAM'].eq("LA Lakers"),['SAME CITY']  ] = 1
# nba_regular_season.loc[nba_regular_season['OWN TEAM'].eq("New York") & nba_regular_season['OPP TEAM'].eq("Brooklyn"), ['SAME CITY'] ] = 1
# nba_regular_season.loc[nba_regular_season['OWN TEAM'].eq("Brooklyn") & nba_regular_season['OPP TEAM'].eq("New York"),['SAME CITY'] ] = 1
# 
# nba_regular_season['TRAVEL'] = 1
# # Played back to back home games
# nba_regular_season.loc[nba_regular_season['RR VAL'].eq('HH'), ['TRAVEL']] = 0
# # Last game was H, Current is R but in the Same City
# nba_regular_season.loc[nba_regular_season['RR VAL'].eq('HR') & nba_regular_season['SAME CITY'].eq(1), ['TRAVEL']] = 0
# # Last game was R in Same City, Current game is H
# nba_regular_season.loc[nba_regular_season['RR VAL'].eq('RH') & nba_regular_season['SAME CITY'].shift(1).eq(1), ['TRAVEL']] = 0


In [19]:
# Create Dummy Variables for 'REST DAYS'
nba_reg_season_dummies = pd.concat([
    nba_regular_season,
    pd.get_dummies(nba_regular_season['REST DAYS'])
], axis=1)

In [20]:
game = [ 'DATASET',       'DATE',       'VENUE',      'TEAMS',          'F',        
              'MIN',         'FG',        'FGA',         '3P',        '3PA',         
               'FT',        'FTA',         'OR',         'DR',        'TOT',          
                'A',         'PF',         'ST',         'TO',         'BL',        
              'PTS',       'POSS',       'PACE',       'OEFF',       'DEFF' ]

nba_reg_season_stats = nba_regular_season[game].groupby(['TEAMS', 'DATASET'], dropna=True).mean()
nba_reg_season_stats_home_away = nba_regular_season[game].groupby(['TEAMS', 'DATASET','VENUE'], dropna=True).mean()

In [21]:
nba_reg_dates = pd.concat([
    nba_regular_season[game].groupby(['TEAMS', 'DATASET'], dropna=True)['DATE'].min(), 
    nba_regular_season[game].groupby(['TEAMS', 'DATASET'], dropna=True)['DATE'].max(),
], axis = 1 )

nba_reg_season_dummies = pd.concat([
    nba_regular_season,
    pd.get_dummies(nba_regular_season['DATE-DIFF'])
], axis=1)

Season Statistics -> 

In [22]:
with pd.ExcelWriter("../data/BDB_Combo.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    nba_regular_season.to_excel(writer, sheet_name="General RS", index=False)
    nba_reg_dates.to_excel(writer, sheet_name="RS - Date", index=True)
    nba_reg_season_dummies.to_excel(writer, sheet_name= "RS - Dummies", index=False)
    nba_reg_season_stats.to_excel(writer, sheet_name="RS - Avg Season Stats", index=True)
    nba_reg_season_stats_home_away.to_excel(writer, sheet_name="RS - HomeAway Stats", index=True)